# Semi-structured eval: Chunk size tuning

`Semi-structured Reports` is a public dataset that contains question-answer pairs from documents with text and tables.

The question-answer pairs are derived from the tables as well as some of the paragraphs in the docs.

We evaluation performance of various chunk sizes with RAG. 

## Pre-requisites

In [ ]:
# %pip install -U langchain langsmith langchain_benchmarks
# %pip install --quiet chromadb openai pypdf tiktoken fireworks-ai

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
env_vars = ["LANGCHAIN_API_KEY", "OPENAI_API_KEY", "FIREWORKS_API_KEY"]
for var in env_vars:
    if var not in os.environ:
        os.environ[var] = getpass.getpass(prompt=f"Enter your {var}: ")

## Dataset

Fetch the associated PDFs from remote cache for the dataset so that we can perform ingestion.

In [3]:
import os

from langchain_benchmarks import clone_public_dataset, registry
from langchain_benchmarks.rag.tasks.semi_structured_reports import get_file_names

# Task
task = registry["Semi-structured Reports"]

# Files used
paths = list(get_file_names())
files = [str(p) for p in paths]

Clone the dataset so that it's available in our LangSmith datasets.

In [4]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Semi-structured Reports already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c.


## Load and index

We load each file, split it, embed with `OpenAIEmbeddings`, and create an index with `Chroma` vectorstore.

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatFireworks
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


def load_and_split(file, token_count, split_document=True):
    """
    Load and optionally split PDF files.

    Args:
        file (str): File path.
        token_count (int): Token count for splitting.
        split_document (bool): Flag for splitting or returning pages.
    """

    loader = PyPDFLoader(file)
    pdf_pages = loader.load()

    if split_document:
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=token_count, chunk_overlap=50
        )

        docs = text_splitter.split_documents(pdf_pages)
        texts = [d.page_content for d in docs]
    else:
        texts = [d.page_content for d in pdf_pages]

    print(f"There are {len(texts)} text elements")
    return texts


def load_files(files, token_count, split_document):
    """
    Load files.

    Args:
        files (list): List of file names.
        dir (str): Directory path.
        token_count (int): Token count for splitting.
        split_document (bool): Flag for splitting documents.
    """

    texts = []
    for fi in files:
        texts.extend(load_and_split(fi, token_count, split_document))
    return texts


def make_retriever(texts, expt):
    """
    Make vector store.

    Args:
        texts (list): List of texts.
        expt (str): Experiment name.
    """
    vectorstore = Chroma.from_texts(
        texts=texts, collection_name=expt, embedding=OpenAIEmbeddings()
    )
    retriever = vectorstore.as_retriever()
    return retriever


def rag_chain(retriever, llm):
    """
    RAG chain.

    Args:
        retriever: The retriever to use.
        llm: The llm to use.
    """

    # Prompt template
    template = """Answer the question based only on the following context, which can include text and tables:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    if llm == "mixtral":
        model = ChatFireworks(model='accounts/fireworks/models/mixtral-8x7b-instruct',
                              temperature=0)
    else:
        model = ChatOpenAI(temperature=0, model="gpt-4")

    # RAG pipeline
    chain = (
        {
            "context": retriever | (lambda x: "\n\n".join([i.page_content for i in x])),
            "question": RunnablePassthrough(),
        }
        | prompt
        | model
        | StrOutputParser()
    )
    return chain


# Experiment configurations
experiments = [
    (None, False, "page_split-oai", "oai"),
    (50, True, "50_tok_split-oai", "oai"),
    (100, True, "100_tok_split-oai", "oai"),
    (250, True, "250_tok_split-oai", "oai"),
    (250, True, "250_tok_split-mixtral", "mixtral"),
]

# Run
stor_chain = {}
for token_count, split_document, expt, llm in experiments:
    texts = load_files(files, token_count, split_document)
    retriever = make_retriever(texts, expt)
    stor_chain[expt] = rag_chain(retriever, llm)

There are 3 text elements
There are 15 text elements
There are 3 text elements
There are 16 text elements
There are 11 text elements
There are 11 text elements
There are 181 text elements
There are 1454 text elements
There are 197 text elements
There are 521 text elements
There are 284 text elements
There are 287 text elements
There are 59 text elements
There are 77 text elements
There are 67 text elements
There are 285 text elements
There are 146 text elements
There are 130 text elements
There are 15 text elements
There are 27 text elements
There are 17 text elements
There are 74 text elements
There are 41 text elements
There are 38 text elements
There are 15 text elements
There are 27 text elements
There are 17 text elements
There are 74 text elements
There are 41 text elements
There are 38 text elements


## Eval

Run eval onm our dataset, `Semi-structured Reports`.

In [ ]:
import uuid
from langsmith.client import Client
from langchain.smith import RunEvalConfig

# Config
client = Client()
eval_config = RunEvalConfig(
    evaluators=["cot_qa"],
)

# Experiments
chain_map = {
    "page_split": stor_chain["page_split-oai"],
    "baseline-50-tok": stor_chain["50_tok_split-oai"],
    "baseline-100-tok": stor_chain["100_tok_split-oai"],
    "baseline-250-tok": stor_chain["250_tok_split-oai"],
    "baseline-250-tok-mixtral": stor_chain["250_tok_split-mixtral"],
}

# Run evaluation
run_id = uuid.uuid4().hex[:4]
test_runs = {}
for project_name, chain in chain_map.items():
    test_runs[project_name] = client.run_on_dataset(
        dataset_name=task.name,
        llm_or_chain_factory=lambda: (lambda x: x["Question"]) | chain,
        evaluation=eval_config,
        verbose=True,
        project_name=f"{run_id}-{project_name}",
        project_metadata={"chain": project_name},
    )

View the evaluation results for project '3366-page_split' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c/compare?selectedSessions=83f14e01-d41d-4155-8d9f-998489e366b9

View all tests for Dataset Semi-structured Reports at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c
[------------------------------------------------->] 30/30

,output,feedback.COT Contextual Accuracy,error,execution_time,run_id
count,30,30.000000,0,30.000000,30
unique,30,NaN,0,NaN,30
top,"Datadog spent $240,225,000 on research and dev...",NaN,NaN,NaN,846c0275-ef46-4574-b3ed-3d850992949b
freq,1,NaN,NaN,NaN,1
mean,NaN,0.833333,NaN,10.098947,NaN
std,NaN,0.379049,NaN,16.387474,NaN
min,NaN,0.000000,NaN,4.363880,NaN
25%,NaN,1.000000,NaN,5.480481,NaN
50%,NaN,1.000000,NaN,6.497657,NaN
75%,NaN,1.000000,NaN,8.186155,NaN


View the evaluation results for project '3366-baseline-50-tok' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c/compare?selectedSessions=ffc20811-1b3d-4f53-90c8-be0948e47923

View all tests for Dataset Semi-structured Reports at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c
[------------------------------------------------->] 30/30

,output,feedback.COT Contextual Accuracy,error,execution_time,run_id
count,30,30.000000,0,30.000000,30
unique,30,NaN,0,NaN,30
top,The text does not provide information on how m...,NaN,NaN,NaN,df89c113-7cff-4d3d-9a40-9fc9a55957e9
freq,1,NaN,NaN,NaN,1
mean,NaN,0.300000,NaN,6.366559,NaN
std,NaN,0.466092,NaN,3.002384,NaN
min,NaN,0.000000,NaN,3.936561,NaN
25%,NaN,0.000000,NaN,4.972215,NaN
50%,NaN,0.000000,NaN,5.614514,NaN
75%,NaN,1.000000,NaN,6.325984,NaN


View the evaluation results for project '3366-baseline-100-tok' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c/compare?selectedSessions=33724dd2-14fb-4755-ae0b-3e128403271d

View all tests for Dataset Semi-structured Reports at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c
[------------------------------------------------->] 30/30

,output,feedback.COT Contextual Accuracy,error,execution_time,run_id
count,30,30.000000,0,30.000000,30
unique,30,NaN,0,NaN,30
top,The text does not provide information on how m...,NaN,NaN,NaN,13325216-6be8-4779-ad70-0a529d0c80b7
freq,1,NaN,NaN,NaN,1
mean,NaN,0.466667,NaN,9.491136,NaN
std,NaN,0.507416,NaN,17.632267,NaN
min,NaN,0.000000,NaN,3.434882,NaN
25%,NaN,0.000000,NaN,4.655369,NaN
50%,NaN,0.000000,NaN,5.906170,NaN
75%,NaN,1.000000,NaN,6.834664,NaN


View the evaluation results for project '3366-baseline-250-tok' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c/compare?selectedSessions=67752aa5-4d92-4095-99f2-26e161d76d87

View all tests for Dataset Semi-structured Reports at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c
[------------------------------------------------->] 30/30

,output,feedback.COT Contextual Accuracy,error,execution_time,run_id
count,30,30.000000,0,30.000000,30
unique,30,NaN,0,NaN,30
top,"Datadog spent $240,225,000 on research and dev...",NaN,NaN,NaN,42fad1bd-6d77-4f78-ab73-98a026b889bc
freq,1,NaN,NaN,NaN,1
mean,NaN,0.700000,NaN,13.342490,NaN
std,NaN,0.466092,NaN,23.232546,NaN
min,NaN,0.000000,NaN,4.447057,NaN
25%,NaN,0.000000,NaN,5.313019,NaN
50%,NaN,1.000000,NaN,6.097117,NaN
75%,NaN,1.000000,NaN,8.251563,NaN


View the evaluation results for project '3366-baseline-250-tok-mixtral' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c/compare?selectedSessions=b1aa3fef-a7fa-43a9-bdf7-ac4ced6966f5

View all tests for Dataset Semi-structured Reports at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/6549a3a5-1cb9-463f-951d-0166cb9cf45c
[>                                                 ] 0/30